# EAS 4610
## Final Group Project
## Glacial Cycles (Pollard 1982)

### Dennis Frank
### Marcus Morris
### Nathan Warner

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import pandas as pd

# Introduction

# Equations

### Equation 1

In [13]:
#Constants
a=0.81e-3 #Snowfall Constant, Unit:(1/yr) 
b=0.3e06 #Ice Melt Constant, Unit:(1/(m*yr))
k=25 #Insolation Sensitivity
S=0 #Elevation of Current Sea Level
pi=917 #ice density (kg/m^3)
pw=1027 #sea water density (kg/m^3)

In [20]:
#Function for Calculating the Mass Balance, G: (Snowfall - Ice Melt)
def G(h,h_prime,h_prime_next,E0,dQ): #h:ice height, h':bedrock elevation, x:distance, E0:current equilibrium altitude, dq:insolation change @ 65°N
    E = E0 + k*dQ
    H = h+h_prime #Surface Elevation: H=h+h'
    if (pi*h < (pw*(S-h_prime))) and (h_prime_next < S): #Calving Conditional Statement (maybe put this in separte fxn)
        return -20
    
    elif (H-E) <= 1500:
        return a*(H-E) - b*(H-E)**2
    
    else:
        return 0.56

In [21]:
t_f = 700000
x_f = 4884
dt = 50 #yr
dx = 55.5 #km
A = 5.77e-4
alpha = 5
beta = 2
nt = int(t_f/dt) + 1
nx = int(x_f/dx) + 1
x = np.linspace(0, x_f, nx)
t = np.linspace(0, t_f, nt)

h_prime = np.zeros(nt)
h = np.zeros(nt)

h_prime[0] = 100
h[0] = 100
h

array([100.,   0.,   0., ...,   0.,   0.,   0.])

In [23]:
halpha = h**alpha
term1 = (abs((h + h_prime)/dx))**beta
term2 = ((h + h_prime)/dx)
Gfunction = np.zeros(nt) #placeholder

nt
#Define Insolation as a Function of Time (Q)
df = pd.read_csv('Milankovitch.dat',skiprows=1300,delim_whitespace=True, names=['year','lat','Q','x','y','z','v']) #only first 3 rows are of interest
Q = df['Q'].to_numpy() #values in W/m^2
n = np.ceil(len(t)/len(Q))
Q = list(np.repeat(Q,n)) #repeat values
#Run Loop to calculate dQ
Q_present=Q[-1]
dQ = np.ones((len(Q),1))
for i in np.arange(len(Q)):
    dQ[i] = Q[i] - Q_present
#Define Equilibrium-Line Altitude as a Function of 
E0 = (0.9e-3)*x + 55.5*2
for i in np.arange(0, nt - 1):
    h[i+1] = (A * ((halpha[i]) * term1[i] * term2[i])/dx) + G(h[i],h_prime[i],h_prime[i+1],E0,dQ[i])
    i += 1
term2[0]


<ipython-input-23-53c5a5913668>:1: RuntimeWarning: overflow encountered in power
  halpha = h**alpha
<ipython-input-23-53c5a5913668>:2: RuntimeWarning: overflow encountered in square
  term1 = (abs((h + h_prime)/dx))**beta
<ipython-input-23-53c5a5913668>:20: RuntimeWarning: overflow encountered in double_scalars
  h[i+1] = (A * ((halpha[i]) * term1[i] * term2[i])/dx) + G(h[i],h_prime[i],h_prime[i+1],E0[i],dQ[i])


IndexError: index 89 is out of bounds for axis 0 with size 89

In [32]:
len(d)


89

### Equation 2: Net Annual Ice Surface Mass Balance

(write out equation here)

In [89]:
#Define Insolation as a Function of Time (Q)
df = pd.read_csv('Milankovitch.dat',skiprows=1300,delim_whitespace=True, names=['year','lat','Q','x','y','z','v']) #only first 3 rows are of interest
Q = df['Q'].to_numpy() #values in W/m^2
n = np.ceil(len(t)/len(Q))
Q = list(np.repeat(Q,n)) #repeat values

#Run Loop to calculate dQ
Q_present=Q[-1]
dQ = np.ones((len(Q),1))
for i in np.arange(len(Q)):
    dQ[i] = Q[i] - Q_present

#Define Equilibrium-Line Altitude as a Function of 
E0 = (0.9e-3)*x + 55.5*2

### Equation 3

In [20]:
r = 0.3
nu = 100 # km^2/yr
hprime0 = 0

t_f = 700000
x_f = 4884
dt = 50 #yr
dx = 55.5 #km
nt = int(t_f/dt) + 1
nx = int(x_f/dx) + 1
x = np.linspace(0, x_f, nx)
t = np.linspace(0, t_f, nt)

c_nu = (nu * dt) / (dx**2)

h_prime = np.zeros([nx, nt])
h_0 = np.zeros([nx]) ##initialize
h = np.zeros([nx, nt]) #placeholder

h_prime_data = np.array([c_nu*np.ones(nx), (1-2*c_nu)*np.ones(nx), c_nu*np.ones(nx)])
h_prime_diags = np.array([-1, 0, 1])
h_prime_M = spdiags(h_prime_data, h_prime_diags, nx, nx)

h_0_data = np.array([-c_nu*np.ones(nx), (2*c_nu)*np.ones(nx), -c_nu*np.ones(nx)])
h_0_diags = np.array([-1, 0, 1])
h_0_M = spdiags(h_0_data, h_0_diags, nx, nx)

h_data = np.array([r*c_nu*np.ones(nx), (-2*r*c_nu)*np.ones(nx), r*c_nu*np.ones(nx)])
h_diags = np.array([-1, 0, 1])
h_M = spdiags(h_data, h_diags, nx, nx)

## Boundary conditions

## Initial conditions

for k in np.arange(0, nt - 1):
    h_prime_new = np.matmul(h_prime_M, h_prime[:,k]) + np.matmul(h_0_M, h_0[:]) + np.matmul(h_M, h[:,k])
    h[:, k+1] = h_prime_new.transpose()

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

# Numerical Methods

# Results

# Figures

# Video Walkthrough

# References